In [1]:
import cupy as np # Use Cuda
from h5py import File
from PIL import Image
from sandbox import model, layers, activations, costs, predictions

np.random.seed(1)

In [2]:
# Load data
train_dataset = File('dataset\\train_catvnoncat.h5', 'r')
train_x = np.array(train_dataset['train_set_x'][:]) # Train set features
train_y = np.array(train_dataset['train_set_y'][:]) # Train set labels

test_dataset = File('dataset\\test_catvnoncat.h5', 'r')
test_x = np.array(test_dataset['test_set_x'][:]) # Test set features
test_y = np.array(test_dataset['test_set_y'][:]) # Test set labels

train_y = train_y.reshape((train_y.shape[0], 1))
test_y = test_y.reshape((test_y.shape[0], 1))

# Flatten and normalize
train_x = train_x.reshape(train_x.shape[0], -1) / 255
test_x = test_x.reshape(test_x.shape[0], -1) / 255

In [4]:
cat = model.Model(cuda=True)
cat.add(layers.Dense(units=20, activation=activations.ReLU()))
cat.add(layers.Dense(units=7, activation=activations.ReLU()))
cat.add(layers.Dense(units=5, activation=activations.ReLU()))
cat.add(layers.Dense(units=1, activation=activations.Sigmoid()))

cat.configure(learning_rate=0.0075, epochs=1500, cost_type=costs.BinaryCrossentropy())
cat.train(train_x, train_y, verbose=True)
cat.save() # Save model parameters

[20, 7, 5, 1]
[{'W': array([[-1.88223867e-02, -4.44171946e-03,  1.85922233e-02, ...,
        -5.45797313e-03,  9.14719285e-03,  1.22952398e-02],
       [-9.59483081e-03,  2.74489474e-03, -8.40953804e-04, ...,
         8.91158147e-05,  4.74051701e-03, -1.06353969e-02],
       [-6.44320941e-03, -7.31270485e-03,  6.49612012e-03, ...,
         3.25185261e-03,  1.52838624e-02, -2.31209397e-03],
       ...,
       [ 7.39790855e-03, -1.97431660e-03, -2.46308690e-03, ...,
         1.78100924e-03,  1.02959100e-02, -1.67168550e-02],
       [ 4.99072558e-03,  7.75382312e-03,  5.53264776e-03, ...,
        -1.43296150e-02,  7.70583896e-03, -1.03508953e-02],
       [ 9.76033569e-03, -1.03306743e-02, -8.32923916e-03, ...,
         1.83060350e-02, -5.82482901e-03,  2.77320694e-03]]), 'b': array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [

In [ ]:
# Assess model accuracy
pred_train = cat.predict(train_x, prediction_type=predictions.binary_classification) # Get model accuracy on training data
print('\nTraining Accuracy: '  + str(np.round(np.sum((pred_train == train_y)/train_x.shape[0]), decimals=5)))
pred_test = cat.predict(test_x, prediction_type=predictions.binary_classification) # Get model accuracy on testing data
print('Testing Accuracy: '  + str(np.round(np.sum((pred_test == test_y)/test_x.shape[0]), decimals=5)))

# Use model on custom image
my_image = 'dataset\\cat.jpg' 
num_px = 64

image = np.array(Image.open(my_image).resize((num_px, num_px))) / 255 # Resize and normalize image, cast to NumPy array
image = image.reshape((1, num_px * num_px * 3)) # Flatten image array
my_predicted_image = cat.predict(image, prediction_type=predictions.binary_classification) # Predict custom image
print(f"Image guess: {my_predicted_image.item()}")